In [2]:
import os

In [3]:
%pwd

'c:\\Users\\shailly\\Text-Summarizer\\research'

In [4]:
os.chdir("../")

In [51]:
%pwd

'c:\\Users\\shailly\\Text-Summarizer'

In [49]:
# this is my entity that is written of my function
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [56]:
%pip install ensure_annotations
from box.exceptions import BoxValueError
from box import ConfigBox
%pip install ensure_annotations
from ensure import ensure_annotations
from pathlib import Path
import yaml
from textSummarizer.logging import logger
CONFIG_FILE_PATH = Path("config/config.yaml")
PARAMS_FILE_PATH = Path("params.yaml")

@ensure_annotations
def read_yaml(path_to_yaml: Path) -> ConfigBox:
    """reads yaml file and returns

    Args:
        path_to_yaml (str): path like input

    Raises:
        ValueError: if yaml file is empty
        e: empty file

    Returns:
        ConfigBox: ConfigBox type
    # Add this line to define the logger
    logger = logging.getLogger(__name__)

    # Rest of the code...
    """
    try:
        with open(path_to_yaml) as yaml_file:
            content = yaml.safe_load(yaml_file)
            logger.info(f"yaml file: {path_to_yaml} loaded successfully")
            return ConfigBox(content)
    except BoxValueError:
        raise ValueError("yaml file is empty")
    except Exception as e:
        raise e

@ensure_annotations
def create_directories(path_to_directories: list, verbose=True):
    """create list of directories

    Args:
        path_to_directories (list): list of path of directories
        ignore_log (bool, optional): ignore if multiple dirs is to be created. Defaults to False.
    """
    for path in path_to_directories:
        os.makedirs(path, exist_ok=True)
        if verbose:
            logger.info(f"created directory at: {path}")


Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement ensure_annotations (from versions: none)
ERROR: No matching distribution found for ensure_annotations


Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement ensure_annotations (from versions: none)
ERROR: No matching distribution found for ensure_annotations


In [57]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])
    
    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        
        create_directories([config.root_dir])
        
        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )
        return data_ingestion_config

In [59]:
import os
import urllib.request as request
import zipfile
from textSummarizer.logging import logger
@ensure_annotations
def get_size(path: Path) -> str:
    """get size in KB

    Args:
        path (Path): path of the file

    Returns:
        str: size in KB
    """
    size_in_kb = round(os.path.getsize(path)/1024)
    return f"~ {size_in_kb} KB"




In [61]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
        
        
    def download_data(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url=self.config.source_URL,
                filename=self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")
            
    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
        

In [63]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_data()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-08-01 15:23:55,952: INFO: 1672190045: yaml file: config\config.yaml loaded successfully]
[2024-08-01 15:23:55,955: INFO: 1672190045: yaml file: params.yaml loaded successfully]
[2024-08-01 15:23:55,962: INFO: 1672190045: created directory at: artifacts]
[2024-08-01 15:23:55,964: INFO: 1672190045: created directory at: artifacts/data_ingestion]
[2024-08-01 15:24:01,018: INFO: 1593102423: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 7903594
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "dbc016a060da18070593b83afff580c9b300f0b6ea4147a7988433e04df246ca"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: ED10:29E3AF:4779F6:5A9EC7:66AB5B39
Accept-Ranges: bytes
Date: Thu, 01 Aug 2024 09:54:02 GMT
Via: 1.1 varnish
X-Served-By: cache-maa1023